<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Открытие-файла" data-toc-modified-id="Открытие-файла-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Открытие файла</a></span></li><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Классификация-деревом-решений" data-toc-modified-id="Классификация-деревом-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Классификация деревом решений</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Определение позитивных и негативных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. 

## Подготовка

### Импорт библиотек

In [1]:
import nltk
import pandas as pd
import re


from catboost import CatBoostClassifier
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

### Открытие файла

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Изучение данных

Столбец text - текст комментария, toxic — целевой признак.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Датасет содержит 159571 строку, большая часть текстов комментариев позитивные.

In [5]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

В столбце toxic присутствует дисбаланс классов.

### Лемматизация

In [6]:
m = nltk.WordNetLemmatizer()

def lemmatize(text):
    re_text = re.sub(r'[^a-zA-Z]', ' ', text)
    word_list = nltk.word_tokenize(re_text)
    lemmatized = " ".join([m.lemmatize(word) for word in word_list])
    return " ".join(lemmatized.split())

data['text_lem'] = data['text'].apply(lemmatize)

In [7]:
data.head()

,text,toxic,text_lem
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


In [8]:
data = data.drop(['text'], axis=1)

### Разделение на выборки

In [9]:
target = data['toxic']
features = data['text_lem']

In [10]:
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=12345)

In [11]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
features_train = count_tf_idf.fit_transform(features_train.values)
features_valid = count_tf_idf.transform(features_valid.values)
features_test = count_tf_idf.transform(features_test.values)
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(95742, 121737)
(31914, 121737)
(31915, 121737)


### Вывод

Датасет содержит 159571 строку, большая часть текстов комментариев позитивные.

Столбец text - текст комментария, toxic — целевой признак.

В столбце toxic присутствует дисбаланс классов. При обучении моделей при подборе гиперпараметров учтем его.

Проведена леметизация и раделение на выборки.

## Обучение

### Логистическая регрессия

In [17]:
model_lr = LogisticRegression()


hyperparams = [{'solver':['newton-cg', 'lbfgs', 'liblinear','sag','saga'],
                'C':[0.1, 1, 4, 10, 50, 100, 120],
                'fit_intercept':[True, False],
                'class_weight':['balanced', None]}]

grid = GridSearchCV(model_lr, hyperparams, scoring='f1', cv=3)
grid.fit(features_train, target_train)

lr_best_params = grid.best_params_

print("Лучшие параметры логистической регрессии:", lr_best_params)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

Лучшие параметры логистической регрессии: {'C': 10, 'class_weight': 'balanced', 'fit_intercept': True, 'solver': 'newton-cg'}


In [28]:
model_lr = LogisticRegression(random_state=12345, class_weight='balanced', C=10, fit_intercept=True, solver='newton-cg')
model_lr.fit(features_train, target_train)
predictions_lr = model_lr.predict(features_valid)

print('F1-мера модели логистической регрессии:', f1_score(target_valid, predictions_lr))

F1-мера модели логистической регрессии: 0.7597749242533546


F1-мера модели логистической регрессии: 0.75

### Классификация деревом решений

In [29]:
model_dtc = DecisionTreeClassifier()


hyperparams = [{'max_depth':[x for x in range(65,81,5)],
                'class_weight':['balanced', None]}]

grid = GridSearchCV(model_dtc, hyperparams, scoring='f1', cv=3)
grid.fit(features_train, target_train)

lr_best_params = grid.best_params_

print("Лучшие параметры классификации деревом решений:", lr_best_params)

Лучшие параметры классификации деревом решений: {'class_weight': None, 'max_depth': 75}


In [30]:
model_dtc = DecisionTreeClassifier(random_state=12345, class_weight=None, max_depth=75)
model_dtc.fit(features_train, target_train)
predictions_dtc = model_dtc.predict(features_valid)

print('F1-мера модели классификации деревом решений:', f1_score(target_valid, predictions_dtc))

F1-мера модели классификации деревом решений: 0.7118994317203376


F1-мера модели классификации деревом решений: 0.71

### CatBoostClassifier

In [31]:
model_cat = CatBoostClassifier(verbose=False, iterations=300)
model_cat.fit(features_train, target_train)
predictions_cat = model_cat.predict(features_valid)

print('F1-мера модели CatBoostClassifier:', f1_score(target_valid, predictions_cat))

F1-мера модели CatBoostClassifier: 0.7482040229885057


F1-мера модели CatBoostClassifier: 0.74

### Вывод

Нами было обучено три модели:

- Логистическая регрессия
- Классификация деревом решений
- CatBoostClassifier

Лучшая F1-мера из обученных моделей у логистической регрессии и CatBoostClassifier. Передадим им тестовые данные и посмотрим на результат.

## Тестирование

In [32]:
predictions_lr_test = model_lr.predict(features_test)

print('F1-мера модели логистической регрессии:', f1_score(target_test, predictions_lr_test))

F1-мера модели логистической регрессии: 0.7594788464353681


In [33]:
predictions_cat_test = model_cat.predict(features_test)

print('F1-мера CatBoostClassifier:', f1_score(target_test, predictions_cat_test))

F1-мера CatBoostClassifier: 0.742305590967037


## Выводы

Нами были импортированы библиотеки, открыт и изучен файл.

Датасет содержит 159571 строку, большая часть текстов комментариев позитивные.

Столбец text - текст комментария, toxic — целевой признак.

В столбце toxic присутствует дисбаланс классов, мы учли это при обучении моделей и подборе гиперпараметров.

Проведена леметизация и раделение на выборки.

Нами было обучено три модели:

- Логистическая регрессия
- Классификация деревом решений
- CatBoostClassifier

Лучшая F1-мера из обученных моделей оказалась у логистической регрессии. Передав ей тестовые данные мы получили F1-меру: 0.759.